In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/paraphrase/parasci/test.src.truecase.translated
# !wget https://f000.backblazeb2.com/file/malay-dataset/paraphrase/parasci/test.tgt.truecase.translated
# !wget https://f000.backblazeb2.com/file/malay-dataset/paraphrase/parasci/test.src.1.truecase.translated
# !wget https://f000.backblazeb2.com/file/malay-dataset/paraphrase/parasci/test.tgt.1.truecase.translated

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [3]:
import sentencepiece as spm
sp_model = spm.SentencePieceProcessor()
sp_model.Load('prepare/sp10m.cased.ms-en.model')

True

In [4]:
import tensorflow as tf
import tensorflow_text
import struct

unknown = b'\xff\xff\xff\xff'

def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    for node in graph_def.node:
        
        if node.op == 'RefSwitch':
          node.op = 'Switch'
          for index in xrange(len(node.input)):
            if 'moving_' in node.input[index]:
              node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
          node.op = 'Sub'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'AssignAdd':
          node.op = 'Add'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'Assign':
          node.op = 'Identity'
          if 'use_locking' in node.attr: del node.attr['use_locking']
          if 'validate_shape' in node.attr: del node.attr['validate_shape']
          if len(node.input) == 2:
            node.input[0] = node.input[1]
            del node.input[1]
            
        if 'Reshape/shape' in node.name or 'Reshape_1/shape' in node.name:
            b = node.attr['value'].tensor.tensor_content
            arr_int = [int.from_bytes(b[i:i + 4], 'little') for i in range(0, len(b), 4)]
            if len(arr_int):
                arr_byte = [unknown] + [struct.pack('<i', i) for i in arr_int[1:]]
                arr_byte = b''.join(arr_byte)
                node.attr['value'].tensor.tensor_content = arr_byte
            
            if len(node.attr['value'].tensor.int_val):
                node.attr['value'].tensor.int_val[0] = -1
    
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [5]:
parasci_src = ['test.src.truecase.translated', 
               'test.src.1.truecase.translated']
parasci_tgt = ['test.tgt.truecase.translated', 
               'test.tgt.1.truecase.translated']

In [6]:
import json

X, Y = [], []
directory = './'
for i in range(len(parasci_src)):
    file_src = os.path.join(directory, parasci_src[i])
    file_tgt = os.path.join(directory, parasci_tgt[i])
    with open(file_src) as fopen:
        left = fopen.read().split('\n')
    with open(file_tgt) as fopen:
        right = fopen.read().split('\n')
    if len(left) < len(right):
        right = right[:len(left)]
    else:
        left = left[:len(right)]
        
    x, y = [], []
    for k in range(len(left)):
        if len(left[k]) and len(right[k]):
            l_left = json.loads(left[k])
            l_right = json.loads(right[k])
            x.append(l_left['ms'])
            y.append(l_right['ms'])
            x.append(l_right['ms'])
            y.append(l_left['ms'])
            
    print(len(x), len(y))
    X.extend(x)
    Y.extend(y)

4690 4690
5098 5098


In [7]:
g = load_graph('small/frozen_model.pb')
x = g.get_tensor_by_name('import/inputs:0')
logits = g.get_tensor_by_name('import/SelectV2_3:0')
test_sess = tf.InteractiveSession(graph = g)

In [17]:
from tqdm import tqdm

batch_size = 10
results = []
for i in tqdm(range(0, len(X), batch_size)):
    batch_x = X[i: i + batch_size]
    batches = []
    for b in batch_x:
        batches.append(f'parafrasa: {b}')
    g = test_sess.run(logits, feed_dict = {x:batches})
    results.extend(g.tolist())

100%|██████████| 979/979 [12:53<00:00,  1.27it/s]


In [32]:
results_Y = [sp_model.DecodeIds(r) for r in results]
results_Y[0], Y[0]

('Kami menjalankan Mecab pada Hadoop 11, perisian sumber terbuka yang melaksanakan rangka kerja Map-Reduce, untuk perkataan Segmenting dan Pos yang menandai data.',
 'Kami menjalankan Mecab 4 dengan kamus Ipa 5 pada Hadoop 6, perisian sumber terbuka yang melaksanakan rangka kerja Map-Reduce, untuk pensijilan kata selari, penandaan Part-Of-Speech, dan Kana Pronunciation Annotating.')

In [28]:
from tensor2tensor.utils import bleu_hook

In [33]:
bleu_hook.compute_bleu(reference_corpus = Y, 
                       translation_corpus = results_Y)

0.6174561